In [1]:
import pandas as pd
import numpy as np
import re
import requests

In [2]:
df = pd.read_csv('raw.csv')

pattern = r'(?<=\.vn)/([^/]+/[^/]+/)'
df['data'] = df['link'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else '')
print(df.head())


                                                link           price    area  \
0  https://mogi.vn/quan-thanh-khe/mua-nha-hem-ngo...  3 tỷ 590 triệu   41 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-dat-tho-...  4 tỷ 399 triệu  115 m2   
2  https://mogi.vn/quan-hai-chau/mua-nha-hem-ngo/...  2 tỷ 690 triệu   80 m2   
3  https://mogi.vn/quan-son-tra/mua-nha-biet-thu-...  7 tỷ 190 triệu  123 m2   
4  https://mogi.vn/quan-son-tra/mua-dat-tho-cu/gi...            5 tỷ  100 m2   

   room toilet                                            address  \
0  4 PN   3 WC  Điện Biên Phủ, Phường Chính Gián, Quận Thanh K...   
1  0 PN   0 WC  Nguyễn Văn Thoại, Bắc Mỹ An, Quận Ngũ Hành Sơn...   
2  2 PN   1 WC  Hà Huy Giáp, Phường Hòa Cường Nam, Quận Hải Ch...   
3  5 PN   4 WC  Vũng Thùng 3, Nại Hiên Đông, Quận Sơn Trà, Đà ...   
4  0 PN   0 WC  An Thượng, Phường Phước Mỹ, Quận Sơn Trà, Đà Nẵng   

                                     data  
0         quan-thanh-khe/mua-nha-hem-ngo/  
1       quan-ngu

In [3]:
# Split the "data" column into two columns based on the slash separator
df[['quan', 'loai_bds']] = df['data'].str.split('/', n=1, expand=True)

# Drop the original "data" column
df.drop('data', axis=1, inplace=True)
print(df.head())

                                                link           price    area  \
0  https://mogi.vn/quan-thanh-khe/mua-nha-hem-ngo...  3 tỷ 590 triệu   41 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-dat-tho-...  4 tỷ 399 triệu  115 m2   
2  https://mogi.vn/quan-hai-chau/mua-nha-hem-ngo/...  2 tỷ 690 triệu   80 m2   
3  https://mogi.vn/quan-son-tra/mua-nha-biet-thu-...  7 tỷ 190 triệu  123 m2   
4  https://mogi.vn/quan-son-tra/mua-dat-tho-cu/gi...            5 tỷ  100 m2   

   room toilet                                            address  \
0  4 PN   3 WC  Điện Biên Phủ, Phường Chính Gián, Quận Thanh K...   
1  0 PN   0 WC  Nguyễn Văn Thoại, Bắc Mỹ An, Quận Ngũ Hành Sơn...   
2  2 PN   1 WC  Hà Huy Giáp, Phường Hòa Cường Nam, Quận Hải Ch...   
3  5 PN   4 WC  Vũng Thùng 3, Nại Hiên Đông, Quận Sơn Trà, Đà ...   
4  0 PN   0 WC  An Thượng, Phường Phước Mỹ, Quận Sơn Trà, Đà Nẵng   

                quan                   loai_bds  
0     quan-thanh-khe           mua-nha-hem-ngo/  
1  q

In [4]:
def convert_to_dong(string):
    if not isinstance(string, str):
        return 0
    pattern = r'(\d+)\s*(tỷ)?\s*(\d+)?\s*(triệu)?'
    string = string.replace('.', '')
    match = re.search(pattern, string)
    
    if match.group(2) and match.group(4):
        return int(match.group(1)) * 1000000000 + int(match.group(3)) * 1000000
    elif match.group(2):
        return int(match.group(1)) * 1000000000
    elif match.group(4):
        return int(match.group(1)) * 1000000
    else:
        return int(match.group(1))

def convert_to_number(string):
    numbers = re.findall(r'\d+', string)
    return numbers[0]

In [5]:
print(df.isna().sum())
print(convert_to_dong('5 tỷ'))

link        0
price       2
area        0
room        0
toilet      0
address     0
quan        0
loai_bds    0
dtype: int64
5000000000


In [6]:

df['price'].fillna(0)
df['price-VND'] = df['price'].apply(convert_to_dong)
df = df.sort_values('price-VND', ascending=True)
df['room']=df['room'].apply(convert_to_number).astype('Int64')
df['toilet']=df['toilet'].apply(convert_to_number).astype('Int64')
df['area']=df['area'].apply(convert_to_number).astype('Int64')


In [7]:
df

,link,price,area,room,toilet,address,quan,loai_bds,price-VND
5011,https://mogi.vn/quan-lien-chieu/mua-dat-nen-du...,NaN,100,0,0,"Nguyễn Tất Thành, Phường Hoà Hiệp Nam, Quận Li...",quan-lien-chieu,mua-dat-nen-du-an/,0
7107,https://mogi.vn/quan-lien-chieu/mua-dat-nen-du...,NaN,100,0,0,"Nguyễn Lương Bằng, Phường Hoà Hiệp Nam, Quận L...",quan-lien-chieu,mua-dat-nen-du-an/,0
335,https://mogi.vn/quan-hai-chau/mua-nha-mat-tien...,100.000 đ,120,4,5,"Bạch Đằng, Phường Hải Châu I, Quận Hải Châu, Đ...",quan-hai-chau,mua-nha-mat-tien-pho/,100000
336,https://mogi.vn/quan-hai-chau/mua-nha-biet-thu...,100.000 đ,120,4,6,"Hoá Sơn 10, Phường Hòa Cường Nam, Quận Hải Châ...",quan-hai-chau,mua-nha-biet-thu-lien-ke/,100000
4244,https://mogi.vn/quan-son-tra/mua-nha-mat-tien-...,105.000 đ,81,7,7,"Thạch Lam, Phường Phước Mỹ, Quận Sơn Trà, Đà Nẵng",quan-son-tra,mua-nha-mat-tien-pho/,105000
...,...,...,...,...,...,...,...,...,...
4220,https://mogi.vn/quan-hai-chau/mua-nha-mat-tien...,5500 tỷ,76,3,2,"Nguyễn Thi, Phường Hòa Cường Nam, Quận Hải Châ...",quan-hai-chau,mua-nha-mat-tien-pho/,5500000000000
2073,https://mogi.vn/quan-cam-le/mua-nha-mat-tien-p...,6992 tỷ,92,3,2,"Tú Mỡ, Phường Hòa An, Quận Cẩm Lệ, Đà Nẵng",quan-cam-le,mua-nha-mat-tien-pho/,6992000000000
2060,https://mogi.vn/quan-thanh-khe/mua-nha-mat-tie...,7800 tỷ,100,4,2,"Thanh Huy 1, Phường Thanh Khê Đông, Quận Thanh...",quan-thanh-khe,mua-nha-mat-tien-pho/,7800000000000
2051,https://mogi.vn/quan-ngu-hanh-son/mua-nha-mat-...,14798 tỷ,151,3,2,"Nguyễn Văn Thoại, Phường Mỹ An, Quận Ngũ Hành ...",quan-ngu-hanh-son,mua-nha-mat-tien-pho/,14798000000000


In [8]:
df.to_csv('clean.csv',index=False)